# Xception

paper: https://arxiv.org/pdf/1610.02357.pdf


In [1]:
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D

In [2]:
def conv_batchnorm(x, filters, kernel_size, strides=1):
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    return x

In [3]:
def separable_conv_batchnorm(x, filters, kernel_size, strides=1):
    x = SeparableConv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    return x

In [5]:
def entry_flow(x):
    x = conv_batchnorm(x, filters=32, kernel_size=3, strides=2)
    x = ReLU()(x)
    x = conv_batchnorm(x, filters=62, kernel_size=3)
    tensor = ReLU()(x)
    
    x = separable_conv_batchnorm(tensor, filters=128, kernel_size=3)
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=128, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2,padding='same')(x)
    
    tensor = conv_batchnorm(tensor, filters=128, kernel_size=1, strides=2)
    
    x = Add()([tensor, x])
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=256, kernel_size=3)
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2,padding='same')(x)
        
    tensor = conv_batchnorm(tensor, filters=256, kernel_size=1, strides=2)
    x = Add()([tensor, x])
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=728, kernel_size=3)
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=728, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2,padding='same')(x)
    
    tensor = conv_batchnorm(tensor, filters=728, kernel_size=1, strides=2)
    x = Add()([tensor, x])
    
    return x

In [11]:
def middle_flow(tensor):
    for _ in range(8):
        x = ReLU()(tensor)
        x = separable_conv_batchnorm(x, filters = 728, kernel_size=3)
        x = ReLU()(x)
        x = separable_conv_batchnorm(x, filters = 728, kernel_size=3)
        x = ReLU()(x)
        x = separable_conv_batchnorm(x, filters = 728, kernel_size=3)
        tensor = Add()([tensor, x])
    return tensor

In [16]:
def exit_flow(tensor):
    x = ReLU()(tensor)
    x = separable_conv_batchnorm(x, filters=728, kernel_size=3)
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=1024, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor =conv_batchnorm(tensor, filters=1024, kernel_size=1, strides=2)
    x = Add()([tensor, x])
    
    x = separable_conv_batchnorm(x, filters=1536, kernel_size=3)
    x = ReLU()(x)
    x = separable_conv_batchnorm(x, filters=2048, kernel_size=3)
    x = ReLU()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(units=1000, activation='softmax')(x)
    
    return x

In [17]:
input_layer = Input(shape=(299, 299, 3))
x = entry_flow(input_layer)
x = middle_flow(x)
output = exit_flow(x)

In [18]:
from tensorflow.keras import Model
model = Model(inputs=input, outputs=output)

ValueError: Input tensors to a Model must come from `tf.keras.Input`. Received: <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000025A7E6E6278>> (missing previous layer metadata).